# Example 2: Virtual Community 

Example 2 shows how **pyrecodes** can be used to assess the resilience of interdependent infrastructure systems and housing in a virtual community. This is a toy example to illustrate how the system recovery and interdependencies are captured in iRe-CoDeS.

Please refer to the **pyrecodes** [Example 2 page](https://nikolablagojevic.github.io/pyrecodes/html/usage/examples/example_2.html) for further details.

![VirtualCommunity](./Example%202/Example_2_Community.png)

In [ ]:
from pyrecodes import main

system = main.run('./Example 2/VirtualCommunity_Main.json')

system.calculate_resilience()

In [ ]:
from pyrecodes.plotter.concrete_plotter import ConcretePlotter

plotter_object = ConcretePlotter()
resources_to_plot = ['ElectricPower', 'HighLevelCommunication', 'LowLevelCommunication',  'PotableWater',  'CoolingWater', 'Shelter']
units = ['[MWh/day]', '[E/day]', '[E/day]', '[m^3/day]', '[m^3/day]', '[beds/day]']
plotter_object.plot_supply_demand_dynamics(system, resources_to_plot, units)
plotter_object.plot_gantt_chart(system.components[:20])